In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import multiprocessing as mp
import gc
import datetime
from sklearn.preprocessing import LabelEncoder
import calendar
from scipy.sparse import csr_matrix,hstack
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from lightgbm import plot_importance,LGBMRegressor
from tqdm import tqdm
import pickle
import sys
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import plotly.graph_objects as go
import math
from datetime import datetime, timezone
import pytz

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
def rmsse(train, test, forecast):
    forecast_mse = np.mean((test - forecast)**2, axis=1)
    train_mse = [np.mean((np.diff(np.trim_zeros(row))**2)) for row in train]
    return np.sqrt(forecast_mse / train_mse)

# rmsse_vals = rmsse(train.filter(like="d_").values, test.values, forecast.values)
# print(rmsse_vals)

In [4]:
def separate_features_and_parameters():
  df = pd.read_pickle("/content/gdrive/MyDrive/PredictiveAnalyticsData/output_data/final_train.pkl")
  train_df = df.drop(columns=['id'], axis=1)
  print(f"Training Data Shape: {train_df.shape}")
  y_train = train_df['sales']
  X_train = train_df.drop(columns=['sales'], axis=1)

  df = pd.read_pickle("/content/gdrive/MyDrive/PredictiveAnalyticsData/output_data/final_test.pkl")
  test_df = df.drop(columns=['id'], axis=1)
  print(f"Testing Data Shape: {test_df.shape}")
  y_test = test_df['sales']
  X_test = test_df.drop(columns=['sales'], axis=1)

  df = pd.read_pickle("/content/gdrive/MyDrive/PredictiveAnalyticsData/output_data/final_prediction_data.pkl")
  prediction_df = df.drop(columns=['id'], axis=1)
  print(f"Prediction Data Shape: {prediction_df.shape}")
  y_prediction = prediction_df['sales']
  X_prediction = prediction_df.drop(columns=['sales'], axis=1)

  del df

  return X_train, y_train, X_test, y_test, X_prediction, y_prediction

In [38]:
def reshape(input):
    value = np.reshape(input, (-1, 28),order = 'F')
    return value


def submission_term(val_predict,test_predict):

    val_pred =reshape(val_predict)
    test_pred=reshape(test_predict)

    submission=pd.read_csv(r"/content/gdrive/MyDrive/PredictiveAnalyticsData/sample_submission.csv")

    for ind in tqdm(range(28)):
        submission.iloc[:30490,ind+1]=val_pred[:,ind]
        submission.iloc[30490:,ind+1]=test_pred[:,ind]

    return submission

In [36]:
imp_cols_lgbm = ['d', 'roll_mean_7', 'wday', 'week_num', 'sold_lag_7', 'sold_lag_14',
            'roll_mean_14', 'event_name_1', 'snap',  'sell_price']

In [72]:
global X_train, y_train, X_test, y_test, X_prediction, y_prediction
X_train, y_train, X_test, y_test, X_prediction, y_prediction = separate_features_and_parameters()

Training Data Shape: (58327370, 42)
Testing Data Shape: (853720, 42)
Prediction Data Shape: (853720, 42)


In [75]:
def generate_submission(model_path, take_imp_cols=True):
  with_feature_imp = ""

  today = datetime.now(pytz.timezone('US/Central')).strftime("%d%m%Y_%H%M%S")
  _model = pickle.load(open(f"{model_path}", "rb"))
  if take_imp_cols:
    pred_results_test = _model.predict(X_test[imp_cols_lgbm])
    pred_results_prediction = _model.predict(X_prediction[imp_cols_lgbm])
    with_feature_imp = "with_imp_features"
  else:
    pred_results_test = _model.predict(X_test)
    pred_results_prediction = _model.predict(X_prediction)


  submission_file_path = f"/content/gdrive/MyDrive/PredictiveAnalyticsData/output_data/sample_submission_new_{str(_model).split('(')[0]}_{today}_{with_feature_imp}.csv"
  print(f"Generating submission file for model: \n{str(_model).split('(')[0]} at location {submission_file_path}\n")
  submission1 = submission_term(pred_results_test, pred_results_prediction)
  submission1.to_csv(submission_file_path,index=False)
  print(submission1.shape)

In [78]:
# generate_submission("/content/gdrive/MyDrive/PredictiveAnalyticsData/trained_models/lgbm_feature_reduced_model_lr_0.05_ne_200.pkl")